# Hyperparameter Tuning & GridSearchCV

Welcome, This course covers the fundamentals of hyperparameter tuning, the mechanics of Cross-Validation, and practical implementation using `scikit-learn`'s `GridSearchCV`.

## Module 1: Introduction to Hyperparameters
* **1.1 Parameters vs. Hyperparameters:** Understanding the difference between model-learned parameters (e.g., weights in linear regression) and user-defined hyperparameters (e.g., learning rate, tree depth).
* **1.2 The Need for Tuning:** How hyperparameters affect model performance (Underfitting vs. Overfitting).
* **1.3 The Curse of Dimensionality in Tuning:** Why exhaustive search becomes computationally expensive.

# Examples: Parameters vs Hyperparameters

---

## 1️⃣ Linear Regression

### Parameters:
- The weights ($w_1, w_2, \dots, w_n$)
- The bias ($b$)

The model calculates these automatically during training  
(e.g., using **Ordinary Least Squares**).

### Hyperparameters:
- If using Gradient Descent → the **Learning Rate ($\alpha$)**  

You must set this **before training begins**.

---

## 2️⃣ Decision Trees

### Parameters:
- The actual decision rules at each node  
  (e.g., *"Is Age > 25?"*, *"Is Income < $50k?"*)

The model determines these splits based on the data.

### Hyperparameters:
- **Maximum Depth (`max_depth`)**

You decide beforehand how deep the tree can grow  
(e.g., maximum 5 levels).

---

![Underfitting vs Good Fit vs Overfitting](figure.jpg)

# Understanding Underfitting, Good Fit, and Overfitting

---

## 📉 Left Side – Underfitting (bad on both training and testing)

- **Model is too simple**
- **High bias**
- **Poor performance** on training and testing data
- **Example:** Linear model trying to fit complex data


---

## ✅ Middle – Good Fit (Optimal Model)

- **Balanced bias and variance**
- **Learns patterns properly**
- **Performs well** on unseen data
- **Best generalization**

💡 *This is the goal of training.*

---

## 📈 Right Side – Overfitting  (great on training, bad on testing)

- **Model is too complex**
- **Memorizes training data**
- **High variance**
- **Poor performance** on new data
- **Example:** Deep neural network on small dataset


---

# 1.3 The Curse of Dimensionality in Hyperparameter Tuning

---

## 🎯 The Goal
To understand **why we can't just try every possible hyperparameter** and why tuning can take a lot of time.

---

## 📖 The Core Concept
**The Curse of Dimensionality** means:  
> As you add more hyperparameters to tune, the number of combinations grows **exponentially**.  

Checking every combination (brute force) **quickly becomes impossible**.

---

## 💡 Analogy: The Combination Padlock
- **1 dial (0-9):** 10 tries max  
- **3 dials:** 10 × 10 × 10 = 1,000 tries  
- **5 dials:** 10 × 10 × 10 × 10 × 10 = 100,000 tries  

> Every new dial (hyperparameter) **multiplies the work**.

---

## 🛠️ Mathematical Example

Suppose we tune a **Random Forest Classifier** with these options:

| Hyperparameter | Options |
|----------------|---------|
| n_estimators | 10, 50, 100, 200, 500 (5 options) |
| max_depth | None, 10, 20, 30 (4 options) |
| min_samples_split | 2, 5, 10, 15, 20 (5 options) |


**Total combinations:**  

5 × 4 × 5  = 100 unique models


Using **5-Fold Cross-Validation**:  


100 combinations × 5 folds = 500 model trainings



If **1 model takes 1 minute**, the total time = **8.33 hours**!  
Add one more hyperparameter with 10 options → **83.3 hours (~3, 4 days)**!

---


# Module 2: Cross-Validation (The Foundation)

---

## **2.1 What is Cross-Validation?**  
Cross-validation is a technique to **evaluate how well your model will perform on unseen data**.  

- The simple **train-test split** divides your dataset once into a training set and a test set.  
- Problem: The model’s performance depends heavily on **which data ends up in the training vs test set**.  
- **Cross-validation** solves this by **splitting the data multiple times** and averaging results for a more reliable estimate.  

💡 *Analogy:* Testing a recipe multiple times on different groups of friends instead of relying on just one tasting session.  

---

## **2.2 K-Fold Cross-Validation**  
K-Fold Cross-Validation splits the dataset into **K equal parts (folds)**:

1. Use **K-1 folds** to train the model.  
2. Use the **remaining fold** to test the model.  
3. Repeat this process **K times**, each time using a different fold as the test set.  
4. Average the performance across all K runs → **robust estimate of model performance**.  

- Common choice: **K = 5 or 10**  
- Pros: More reliable than a single train-test split, especially on small datasets  

💡 *Example:* With 5-Fold CV, each fold is tested once → 5 performance scores → take the average.  

---

## **2.3 Stratified K-Fold**  
- Sometimes datasets are **imbalanced** (e.g., 90% class A, 10% class B).  
- Regular K-Fold may split data unevenly → some folds might have very few minority class samples.  
- **Stratified K-Fold** ensures **each fold preserves the class proportion** of the original dataset.  

💡 *Analogy:* Think of making sure each tasting group gets a **balanced mix of cookie types** instead of ending up with only chocolate chip cookies in one group.  

- In scikit-learn:

```python
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Module 3: Introduction to GridSearchCV

---

## **3.1 The Grid Search Concept**

**The Goal:**  
You need to **visualize what a "Grid" means** in hyperparameter tuning. We are **systematically defining a finite set of values** to test.

---

### 📖 Core Concept
Grid Search is an **exhaustive search algorithm**:

- You provide a list of hyperparameters and specific values to test.  
- It builds a **mathematical grid** of all possible combinations.  
- Each combination is evaluated to find the **best hyperparameters**.

---

### 💡 Analogy: Restaurant Combo Meal
Imagine building a combo meal:

| Choice | Options | Number of options |
|--------|--------|-----------------|
| Main   | Burger, Chicken Sandwich, Veggie Wrap | 3 |
| Side   | Fries, Salad | 2 |
| Drink  | Soda, Water | 2 |

- Total meals to try = 3 × 2 × 2 = **12 meals**  
- A **Grid Search** is like trying **every possible combo** to find the best one.

---

### 🛠️ Technical Example (Random Forest)

```python
param_grid = {
    'n_estimators': [10, 50],       # 2 options
    'max_depth': [5, 10, None],     # 3 options
}

## 3.2 How GridSearchCV Works (Grid Search + K-Fold CV)

**The Goal:**  
Must understand that **each combination is evaluated rigorously using Cross-Validation**.

---

### 📖 Core Concept
**GridSearchCV = Grid Search + Cross-Validation**  

- For **each hyperparameter combination**, it runs a **full K-Fold Cross-Validation**.  
- The **best parameters** are the ones with the **highest average score** across all folds.

---

### 💡 Analogy: Taste Test Panel
- One person tasting all meals → **biased opinion**.  
- 5 judges tasting all meals → **average score is fairer**.  
- Similarly, **5-Fold CV evaluates each combination on multiple folds** to ensure reliability.

---

### 🛠️ Step-by-Step
1. Pick **combination #1** from the grid.  
2. Split the training data into **K folds**.  
3. Train on **K-1 folds**, test on the remaining fold.  
4. Repeat **K times** → calculate **average score** for combination #1.  
5. Repeat steps 1–4 for **all combinations**.  
6. Compare all averages → pick the **best combination**.

---

In [3]:

import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score

In [4]:
# Load dataset
digits = load_digits()
digits

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [5]:
digits.data

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [6]:
digits.target


array([0, 1, 2, ..., 8, 9, 8])

In [12]:
X, y = digits.data, digits.target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5, 10],
    'criterion':['gini','entropy']
}

# Initialize RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=7,              # 5-Fold Cross Validation
    n_jobs=-1,         # Use all CPU cores
    verbose=2          # Prints progress
)

# Fit the Grid Search to your training data
grid_search.fit(X_train, y_train)

# Best combination found
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

Fitting 7 folds for each of 36 candidates, totalling 252 fits
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation Score: 0.9784208923920029


In [19]:
# Predict on the test set
y_pred = grid_search.predict(X_test)
y_pred_proba = grid_search.predict_proba(X_test)

# Evaluate the model
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
print("ROC AUC Score on test data: ", roc_auc)
print(y_pred_proba)

ROC AUC Score on test data:  0.9992930613967651
[[0.06       0.01016949 0.         ... 0.01       0.         0.29811905]
 [0.         0.05       0.79714286 ... 0.         0.02285714 0.01      ]
 [0.01       0.2        0.03       ... 0.06       0.58496855 0.02037736]
 ...
 [0.99       0.         0.         ... 0.01       0.         0.        ]
 [0.         0.02090909 0.05194805 ... 0.         0.01500866 0.01252291]
 [0.         0.         0.         ... 0.         0.         0.        ]]
